In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import datetime
from homebrewedFunctions.functions import *
import re

In [2]:
all_keys = pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], low_memory =False).sort_index().keys()
pattern = re.compile(r'(\d+)^NAN(\d+)?$')

all_keys = [k for k in all_keys if not pattern.match(k)]
start_exp = all_keys.index("1EXPENDITURE")
start_debt = all_keys.index("1DEBT OUTSTANDING")
keys_dict = {"Revenue": all_keys[:start_exp],
            "Expenditure": [k for k in all_keys[start_exp:start_debt] if "CAPITAL OUTLAY" not in k] +["1CAPITAL OUTLAY"],
            "Debt": [k for k in all_keys[start_debt:] if "CAPITAL OUTLAY" not in k]}
            
stack_dfs = {"Expenditure": pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], usecols = ["State", "Year", "Format", "1GENERAL REVENUE", "1DEBT OUTSTANDING"] + keys_dict["Expenditure"], low_memory =False).sort_index(),
             "Revenue":pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], usecols = ["State", "Year", "Format", "1EXPENDITURE", "1DEBT OUTSTANDING"] + keys_dict["Revenue"], low_memory =False).sort_index(),
             "Debt":pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], usecols = ["State", "Year", "Format", "1EXPENDITURE", "1GENERAL REVENUE"] + keys_dict["Debt"], low_memory =False).sort_index()}

panel_dfs_dict = {}
for rev_exp in stack_dfs.keys():
    panel_dfs_dict[rev_exp] = {}
    for k in stack_dfs[rev_exp].index.get_level_values(2).unique():
        panel_dfs_dict[rev_exp][k] = stack_dfs[rev_exp][stack_dfs[rev_exp].index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index()
        del panel_dfs_dict[rev_exp][k]["Format"]
    panel_dfs_dict[rev_exp] = {k:panel_dfs_dict[rev_exp][k] for k in ["Local government amount",'State & local government amount', 'State government amount']}

for key, dct in panel_dfs_dict.items():
    for k, df in dct.items():
        for col in df.columns:
            if is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(0)

            else:
                try:
                    print(col, "forced")

                    df[col] = pd.to_numeric(df[col], errors = "coerce").fillna(0).astype(float)
                except:
                    # print(col, "skipped")
                    pass
        if key == "Revenue":
            df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
            df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
            df["1DEFICIT"] = df["1EXPENDITURE"].sub(df["1GENERAL REVENUE"])
        if key == "Debt":
            df["1NET INDEBTEDNESS"] = df["1DEBT OUTSTANDING"].sub(df["1OTHER THAN INSURANCE TRUST FUNDS"])
            
        # warning indicates that copying dataframe will defragment it;
        #  not sure if this actually fixes the problem
        df = df.copy()                
cpi_code = {"CPI":"CPIAUCSL"}
start = datetime.datetime(1947,1,1)
end = datetime.datetime.now()
cpi = gather_data(cpi_code, start, end, freq = "A").reset_index().rename(columns = {"DATE": "Year"})
cpi["Year"] = pd.to_datetime(cpi["Year"].astype(str).str[:4], format = "%Y")
cpi["Year"] = cpi["Year"].astype(str)
cpi.set_index("Year", inplace = True)
cpi["CPI"] = cpi["CPI"].div(cpi["CPI"].iloc[-2]).astype(float)
efnagdp = pd.read_csv("EFNAGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()


def set_plot_dfs(panel_dfs_dict, keys, efnagdp, cpi):
    # keys = ["1" + k for k in keys]
    plot_dfs = {}
    for key in panel_dfs_dict.keys():
        plot_dfs[key] = {}
        plot_dfs[key]["Level"] = panel_dfs_dict[key].mul(10**3).copy()
        plot_dfs[key]["Level"].rename(columns = {k:k.replace("1", "").title() for k in plot_dfs[key]["Level"].keys()}, inplace = True)
    #     plot_dfs[key]["Level"]["Total Income"] = plot_dfs[key]["Level"]["Individual Income"].add(plot_dfs[key]["Level"]["Corporate Income"])
    #     plot_dfs[key]["Level"]["Property and Special Assessments"] = plot_dfs[key]["Level"][["Property", "Special Assessments"]].sum(axis = 1)
    #     plot_dfs[key]["Level"]["Deficit"] = plot_dfs[key]["Level"]["Expenditure"].sub(plot_dfs[key]["Level"]["General Revenue"])
        plot_dfs[key]["Level"]["GDP"] = efnagdp["GDP"]
        plot_dfs[key]["Real Level"] = plot_dfs[key]["Level"].div(cpi["CPI"], level = "Year", axis = 0)
        plot_dfs[key]["Level"]["Population"] = efnagdp["Population"]
        plot_dfs[key]["Percent of General Revenue"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["General Revenue"]).mul(100))    
        plot_dfs[key]["Percent of Expenditure"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Expenditure"]).mul(100))    
        plot_dfs[key]["Percent of GDP"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["GDP"]).mul(100))
        
        plot_dfs[key]["Real Value Per Capita"] = plot_dfs[key]["Real Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Population"]))    
        for k in plot_dfs[key].keys():
            plot_dfs[key][k]["EFNA"] = efnagdp["EFNA"]
    return plot_dfs
plot_dfs = {}

for key in panel_dfs_dict.keys():
    plot_dfs[key] = set_plot_dfs(panel_dfs_dict[key], keys_dict[key], efnagdp, cpi)


import os
for revexp_key in plot_dfs.keys():
    for key in plot_dfs[revexp_key].keys():
        try:
            os.mkdir(f"outputs/{key}")
        except:
            pass

regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
scatter_figs = {}
for revexp_key in plot_dfs.keys():
    for key, p_dfs in plot_dfs[revexp_key].items():
        for p_dfskey, df in p_dfs.items():
            map_figs = {}
            html_path = f"outputs/{key}/ScatterPlots{revexp_key}{key}{p_dfskey}.html"
            create_scatter_dropdown(df, regions_df=regions_df,
                                    filename = html_path, 
                                    show_fig = False)
            for name in df.keys():
                map_figs[name] = create_map(df.reset_index(), name, time_name = "Year")
            combined_map_fig = combine_map_figs(map_figs)
            html_path = f"outputs/{key}/MapPlotsByVariableAndYear{revexp_key}{key}{p_dfskey}.html"
            combined_map_fig.write_html(html_path)

for revexp_key in plot_dfs.keys():
    for key, p_dfs in plot_dfs[revexp_key].items():
        # figs = {k: line_dropdown(dataframe, regions_df) for k, dataframe in p_dfs.items()}
        # fig = dict_of_line_figs_to_dropdown_fig(figs, show_fig = False, use_sliders = True)
        filename = f"outputs/{key}/LinePlotsStateFinances{revexp_key}{key}.html"
        fig = aggregated_line_dropdown(p_dfs, regions_df)
        fig.write_html(filename, config=dict(displayModeBar=True))

1INTERGOVERNMENTAL EXPENDITURE forced
1ASSISTANCE AND SUBSIDIES forced
1INSURANCE BENEFITS AND REPAYMENTS forced
2EDUCATION forced
1HIGHER EDUCATION forced
1ELEMENTARY & SECONDARY forced
1OTHER EDUCATION forced
1LIBRARIES forced
1CASH ASSISTANCE PAYMENTS forced
1VENDOR PAYMENTS forced
2HOSPITALS forced
1EMPLOYMENT SECURITY ADMINISTRATION forced
1VETERANS' SERVICES forced
2AIR TRANSPORTATION (AIRPORTS) forced
2PARKING FACILITIES forced
2SEA AND INLAND PORT FACILITIES forced
1FIRE PROTECTION forced
1CORRECTION forced
2SEWERAGE forced
2SOLID WASTE MANAGEMENT forced
1MISCELLANEOUS COMMERCIAL ACTIVITIES forced
1UTILITY EXPENDITURE forced
2WATER SUPPLY forced
2ELECTRIC POWER forced
2GAS SUPPLY forced
2TRANSIT forced
1LIQUOR STORE EXPENDITURE forced
1INSURANCE TRUST EXPENDITURE forced
2UNEMPLOYMENT COMPENSATION forced
2EMPLOYEE RETIREMENT forced
2WORKERS' COMPENSATION forced
1OTHER INSURANCE TRUST forced
1INTERGOVERNMENTAL EXPENDITURE forced
1ASSISTANCE AND SUBSIDIES forced
1INSURANCE BENEFIT

In [3]:
start_year = 2005
areas = {"Revenue":{"Revenue Source by Government" : ["General Revenue From Own Sources", "From Federal Government", "From State Government", "From Local Governments"],
                    "Taxes": ["Intergovernmental Revenue", 'Property', 'Sales And Gross Receipts',
                              'Individual Income', 'Corporate Income', 'Motor Vehicle License', 'Other Taxes', 
                              'Current Charges', 'Interest Earnings', 'Special Assessments', 'Sale Of Property', 'Other General Revenue']},
        "Expenditure": {
            "Expenditures":[
                "Intergovernmental Expenditure", "Current Operations", "Capital Outlay",
                "Assistance And Subsidies", "Interest On Debt", "Insurance Benefits And Repayments"],
            "Expenditure by Function": [
                    "Intergovernmental Expenditure", "Education", "Libraries", "Public Welfare", "Hospitals", 
                    "Health", "Employment Security Administration", "Veterans' Services",
                    "Highways", "Air Transportation (Airports)", "Parking Facilities", "Sea And Inland Port Facilities",
                    "Police Protection", "Fire Protection", "Correction", "Protective Inspection And Regulation",
                    "Natural Resources", "Parks And Recreation", "Housing And Community Development", "Sewerage",
                    "Solid Waste Management", "Financial Administration","Judicial And Legal", "General Public Buildings",
                    "Other Governmental Administration", "Interest On General Debt", "Miscellaneous Commercial Activities",
                    "Other And Unallocable", "Utility Expenditure", "Liquor Store Expenditure", "Insurance Trust Expenditure"]},
            # "Debt": {""}
}
figs = {}
for revexp_key in areas.keys():
    for key, p_dfs in plot_dfs[revexp_key].items():
        figs[key] = {}
        for form, plot_df in p_dfs.items():# ["Level", "Real Level", "Percent of General Revenue", "Percent of GDP", "Real Value Per Capita"]:
            figs[key][form] = {}
            df = plot_df.copy()
            df.rename(columns = {k:k.replace("1", "").replace("2","").title() for k in df.keys()}, inplace = True)
            df = df.reset_index().melt(id_vars=["State","Year"],
                var_name="Name", 
                value_name="Value",
                )
            df = df.set_index(["State", "Year"])#.round(4)
            states = df.index.get_level_values("State").unique()

            for components_group, components in areas[revexp_key].items():


                figs[key][form][components_group] = {}

                for state in states:
                    plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
                    plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
                    plot_df["Value"] = pd.to_numeric(plot_df["Value"])
                    title_key = f"{components_group} {key}<br>{form}<br>{state}"
                    px_fig = px.area(
                        plot_df, x="Year", y="Value", color="Name", title = title_key)

                    figs[key][form][components_group][state] = px_fig
                figs[key][form][components_group]  = dict_of_figs_to_dropdown_fig(figs[key][form][components_group], 
                                                                                show_fig = False,
                                                                                use_sliders = True)
                figs[key][form][components_group].write_html(f"outputs/{key}/AreaPlots{revexp_key}{key}{form}{components_group}Figs.html")

In [29]:
url = "https://www.fraserinstitute.org/sites/default/files/economic-freedom-of-north-america-2022-for-website-allgovs.xlsx"
storage_options = {'User-Agent': 'Mozilla/5.0'}
efna = pd.read_excel(url, storage_options=storage_options, sheet_name=None)
efnagdp = pd.read_csv("EFNAGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()

In [30]:
income = pd.read_csv("PersonalIncomeStateData/CAINC5N__ALL_AREAS_2001_2022.csv", encoding="latin1", low_memory=False)
income["State"] = income["GeoName"].str.extract(r"([A-Z]{2})")
income["GeoFIPS"] = income["GeoFIPS"].str.replace('"', '')
income['StateFIPS'] = income["GeoFIPS"].str.extract(r"(\d{2})")
income["CountyFIPS"] = income["GeoFIPS"].str[-3:].astype(str)
income["County"] = income["GeoName"].str.split(",").str[0]
income.dropna(subset = ["State"], inplace = True)
income.rename(columns = {"Description": "Name"}, inplace = True)

for year in range(2001, 2023):
    year = str(year)
    income[year] = pd.to_numeric(income[year], errors = "coerce")
    income[year][income["Unit"] == income["Unit"].unique()[0]] = income[year][income["Unit"] == income["Unit"].unique()[0]].mul(10**3)

income["Unit"] = income["Unit"].str.replace("Thousands of dollars", "Dollars")
for orig_str, new_str in {"Thousands of dollars": "Dollars", "thousands of dollars": "Dollars", "dollars": "Dollars"}.items():
    income["Name"] = income["Name"].str.replace(orig_str, new_str)
income["Name"] = income["Name"].str.rstrip()
income_names = income["Name"].unique()

melt_df = income[["State", "County", "Name", "Unit"] + [str(y) for y in range(2001,2023)]].melt(id_vars=["State", "County", "Name", "Unit"], var_name="Year", value_name="Value")
melt_df = melt_df.set_index(["State", "County", "Name", "Year"])[["Value"]]
melt_df["Value"] = pd.to_numeric(melt_df["Value"], errors = "coerce")
state_income = melt_df.groupby(["State", "Year", "Name"]).sum()
state_income["Value"][state_income.index.get_level_values("Name") == "Per capita personal income (Dollars)"] = state_income["Value"][state_income.index.get_level_values("Name") == "Personal income (Dollars)"].values / state_income["Value"][state_income.index.get_level_values("Name") == "Population (persons) 2/"].values
pi_dfs = {"Level": state_income.copy()}
pi_dfs["Percent of Personal Income"] = state_income["Value"].div(state_income["Value"].xs("Personal income (Dollars)", level = "Name")).mul(100)

pi_dfs["Value Per Capita"] = state_income["Value"].div(state_income["Value"].xs("Population (persons) 2/", level = "Name"))

/tmp/ipykernel_29338/2542893016.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_29338/2542893016.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_29338/2542893016.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_29338/2542893016.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [31]:
names = list(pi_dfs["Level"].index.get_level_values("Name").unique())
names = [n for n in names if "    " not in n]
areas = {str(i):{} for i in range(3, -1, -1)}
for num_spaces in areas.keys():
    areas[num_spaces] = [n for n in names if " " * int(num_spaces) in n[:int(num_spaces)]]
    names= [n for n in names if n not in areas[num_spaces] ]
    
figs = {}
for form, plot_df in pi_dfs.items():# ["Level", "Real Level", "Percent of General Revenue", "Percent of GDP", "Real Value Per Capita"]:
    figs[form] = {}
    df = plot_df.copy()

    states = df.index.get_level_values("State").unique()

    for components_group, components in areas.items():


        figs[form][components_group] = {}

        for state in states:
            plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
            plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
            plot_df["Value"] = pd.to_numeric(plot_df["Value"])
            title_key = f"{components_group} {key}<br>{form}<br>{state}"
            px_fig = px.area(
                plot_df, x="Year", y="Value", color="Name", title = title_key)

            figs[form][components_group][state] = px_fig
        figs[form][components_group]  = dict_of_figs_to_dropdown_fig(figs[form][components_group], 
                                                                        show_fig = False,
                                                                        use_sliders = True)
        figs[form][components_group].write_html(f"outputs/PersonalIncome/AreaPlots{key}{form}{components_group}.html")

In [32]:
try:
    os.mkdir("outputs/PersonalIncome")
except:
    pass

pi_dfs = {k:v.reset_index().pivot_table(index=['State', 'Year'], columns='Name', values='Value')[income_names] for k,v in pi_dfs.items()}
regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
filename = f"outputs/PersonalIncome/LinePlotsStatePI.html"
fig = aggregated_line_dropdown(pi_dfs, regions_df)
fig.write_html(filename, config=dict(displayModeBar=True))

In [8]:
efna_description = efna["Index Components"]["The Areas and Components of the Economic Freedom of North America Index"].dropna()
codes = efna_description.str[:3]
for char in [":", "."]: 
    codes = codes.str.replace(char, "")
codes = "EFNA " + codes
efna["Index Components"]["Code"] = codes.str.strip()
efna_description = efna["Index Components"].copy().dropna()
efna_description = efna_description[~(efna_description["Code"].str.contains("i")| efna_description["Code"].str.contains("v"))]
efna_description.rename(columns = {"The Areas and Components of the Economic Freedom of North America Index": "Description"}, inplace = True)
efna_description.set_index("Code", inplace = True)
efna_description.loc["EFNA 2B"] = '2B. Top Marginal Income Tax Rate Attributes'
keys = list(efnagdp.keys())
end_index = keys.index("EFNA 6")
efna_df = efnagdp[keys[3:end_index]]
efna_df = efna_df.dropna(how = "all", axis = 0)
efna_sub_keys = {f"EFNA {i}":[k for k in efna_df.keys() if f"EFNA {i}" in k and "i" not in k] for i in range(1,6)}
for i in range(1,6):
    efna_df[f"EFNA {i}"] = efna_df[efna_sub_keys[f"EFNA {i}"]].mean(axis = 1).round(3)
efna_sub_keys["Major EFNA Components"] = ["EFNA 1", "EFNA 2", "EFNA 3", "EFNA 4", "EFNA 5"]



In [9]:

figs = {}

for key, keys in efna_sub_keys.items():
    if len(keys) > 1:
        components_group = key
        components = keys
        figs[components_group] = {}
        df = efna_df.copy()
        df = df.reset_index().melt(id_vars=["State","Year"],
            var_name="Name", 
            value_name="Value",
            )
        df = df.set_index(["State", "Year"])#.round(4)
        states = df.index.get_level_values("State").unique()



        for state in states:
            plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
            plot_df = plot_df[plot_df["Name"].isin(components)].dropna()#.sort_values("Name")
            plot_df["Name"] = plot_df["Name"].apply(lambda x: efna_description.loc[x]["Description"])
            plot_df["Value"] = pd.to_numeric(plot_df["Value"]).div(len(components))
            title_key = f"{components_group} <br>{state}"
            px_fig = px.area(
                plot_df, x="Year", y="Value", color="Name", title = title_key)

            figs[components_group][state] = px_fig
        figs[components_group]  = dict_of_figs_to_dropdown_fig(figs[components_group], 
                                                                        show_fig = False,
                                                                        use_sliders = True)
        figs[components_group].write_html(f"outputs/EFNA/AreaPlots{components_group}.html")


/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the 

In [10]:
regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
fig = line_dropdown(efna_df.sort_index(axis =1), regions_df)
fig.write_html("outputs/EFNA/EFNALinePlot.html")

/home/jamescaton/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [11]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from pandas.api.types import is_numeric_dtype
# import plotly.express as px
# import plotly.graph_objects as go
# import plotly.io as pio
# from plotly.subplots import make_subplots
# import datetime
# from homebrewedFunctions.functions import *
# stack_dfs = pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], low_memory =False).sort_index()
# panel_dfs_dict = {k:stack_dfs[stack_dfs.index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index() for k in stack_dfs.index.get_level_values(2).unique()}
# panel_dfs_dict = {k:panel_dfs_dict[k] for k in ["Local government amount",'State & local government amount', 'State government amount']}
# # panel_dfs_dict["State government amount"][]


In [12]:
# for key, df in panel_dfs_dict.items():
#     for col in df.columns:
#         if is_numeric_dtype(df[col]):
#             df[col] = df[col].fillna(0)

#         else:
#             try:
#                 df[col] = pd.to_numeric(df[col], errors = "coerce").fillna(0).astype(int)
#             except:
#                 pass
#     df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
#     df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
#     df["1DEFICIT"] = df["1EXPENDITURE"].sub(df["1GENERAL REVENUE"])
#     # warning indicates that copying dataframe will defragment it;
#     #  not sure if this actually fixes the problem
#     df = df.copy()

In [13]:
# cpi_code = {"CPI":"CPIAUCSL"}
# start = datetime.datetime(1947,1,1)
# end = datetime.datetime.now()
# cpi = gather_data(cpi_code, start, end, freq = "A").reset_index().rename(columns = {"DATE": "Year"})
# cpi["Year"] = pd.to_datetime(cpi["Year"].astype(str).str[:4], format = "%Y")
# cpi["Year"] = cpi["Year"].astype(str)
# cpi.set_index("Year", inplace = True)
# cpi["CPI"] = cpi["CPI"].div(cpi["CPI"].iloc[-2]).astype(float)
# cpi


In [14]:
# efnagdp = pd.read_csv("EFNAGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()
# efnagdp[["EFNA", "GDP"]]

In [15]:
# map_keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
#         "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
#         "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', #'TOTAL INCOME',
#         "PROPERTY", "SPECIAL ASSESSMENTS"]
#         #"PROPERTY AND SPECIAL ASSESSMENTS",  
#         # "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	


In [16]:
# plot_dfs = {}
# keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
#         "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
#         "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', 'TOTAL INCOME',
#         "PROPERTY", "SPECIAL ASSESSMENTS", "PROPERTY AND SPECIAL ASSESSMENTS",  
#         "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	
# def set_plot_dfs(panel_dfs_dict, keys, efnagdp, cpi):
#     keys = ["1" + k for k in keys]
#     for key in panel_dfs_dict.keys():
#         plot_dfs[key] = {}
#         plot_dfs[key]["Level"] = panel_dfs_dict[key][keys].mul(10**3).copy()
#         plot_dfs[key]["Level"].rename(columns = {k:k.replace("1", "").title() for k in plot_dfs[key]["Level"].keys()}, inplace = True)
#     #     plot_dfs[key]["Level"]["Total Income"] = plot_dfs[key]["Level"]["Individual Income"].add(plot_dfs[key]["Level"]["Corporate Income"])
#     #     plot_dfs[key]["Level"]["Property and Special Assessments"] = plot_dfs[key]["Level"][["Property", "Special Assessments"]].sum(axis = 1)
#     #     plot_dfs[key]["Level"]["Deficit"] = plot_dfs[key]["Level"]["Expenditure"].sub(plot_dfs[key]["Level"]["General Revenue"])
#         plot_dfs[key]["Level"]["GDP"] = efnagdp["GDP"]
#         plot_dfs[key]["Real Level"] = plot_dfs[key]["Level"].div(cpi["CPI"], level = "Year", axis = 0)
#         plot_dfs[key]["Level"]["Population"] = efnagdp["Population"]
#         plot_dfs[key]["Percent of General Revenue"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["General Revenue"]).mul(100))    
#         plot_dfs[key]["Percent of GDP"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["GDP"]).mul(100))
        
#         plot_dfs[key]["Real Value Per Capita"] = plot_dfs[key]["Real Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Population"]))    
#         for key in plot_dfs.keys():
#             plot_dfs[key]["Level"]["EFNA"] = efnagdp["EFNA"]
#     return plot_dfs
# plot_dfs = set_plot_dfs(panel_dfs_dict, keys, efnagdp, cpi)


In [17]:
# import os
# for key in plot_dfs:
#     try:
#         os.mkdir(f"outputs/{key}")
#     except:
#         pass

In [18]:
# regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
# # scatter_figs = {}
# for key, p_dfs in plot_dfs.items():
#     for p_dfskey, df in p_dfs.items():
#         map_figs = {}
#         html_path = f"outputs/{key}/ScatterPlotsIncomePropertyAssessmentSalesFuelTaxesPctTotalRevenue{key}{p_dfskey}.html"
#         create_scatter_dropdown(df, regions_df=regions_df,
#                                 filename = html_path, 
#                                 show_fig = False)
#         for name in df.keys():
#             map_figs[name] = create_map(df.reset_index(), name, time_name = "Year")
#         combined_map_fig = combine_map_figs(map_figs)
#         html_path = f"outputs/{key}/MapPlotsByVariableAndYear{key}{p_dfskey}.html"
#         combined_map_fig.write_html(html_path)

In [19]:
# plot_dfs = {}
# keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
#         "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" , "INTERGOVERNMENTAL REVENUE",
#         "TAXES", "OTHER TAXES", 'PROPERTY', 'SPECIAL ASSESSMENTS', 'PROPERTY AND SPECIAL ASSESSMENTS', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 
#         'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', "TOTAL INCOME", 'MOTOR VEHICLE LICENSE', 
#         'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 
#         'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 
#         'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 
#         'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS',  'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 
#         'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 
#         'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 'OTHER INSURANCE TRUST REVENUE',"EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]
# plot_dfs = set_plot_dfs(panel_dfs_dict, keys, efnagdp, cpi)

In [20]:
# regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")

# for key, p_dfs in plot_dfs.items():
#     # figs = {k: line_dropdown(dataframe, regions_df) for k, dataframe in p_dfs.items()}
#     # fig = dict_of_line_figs_to_dropdown_fig(figs, show_fig = False, use_sliders = True)
#     filename = f"outputs/{key}/LinePlotsStateFinancesAsPercentRevenuePercentGDPAndPerCapitaFigs{key}.html"
#     fig = line_dropdown(p_dfs, regions_df)
#     fig.write_html(filename, config=dict(displayModeBar=True))

In [21]:
# start_year = 2005
# areas = {"Revenue Source by Government" : ["General Revenue From Own Sources", "From Federal Government", "From State Government", "From Local Governments"],
         
#     #  Omit revenue sources according to distinction of subcomponents in revenue data 
#     #  Some sources additionally needed to be removed as their values perfectly explained
#     #  accounting that sums to greater than 100 percent
     
#          "Taxes": ["Intergovernmental Revenue", 'Property', 'Sales And Gross Receipts', 
#                    'Individual Income', 'Corporate Income', 'Motor Vehicle License', 'Other Taxes', 
#                 #    'Charges And Miscellaneous General  Revenue', 
#                    'Current Charges', 
#                 #    'Education', 'Institutions  Of Higher Education', 'School Lunch Sales (Gross)',
#                 #      'Hospitals', 'Highways', 'Air Transportation (Airports)', 'Parking Facilities', 'Sea And Inland Port Facilities', 'Natural Resources',
#                 #       'Parks And Recreation', 'Housing And Community Development', 'Sewerage', 'Solid Waste Management', 'Other Charges', 
#                     #   'Miscellaneous General Revenue', 
#                       'Interest Earnings', 'Special Assessments', 'Sale Of Property', 'Other General Revenue']}
#                     #   'Utility Revenue']}
#                     #   'Water Supply', 'Electric Power', 'Gas Supply', 'Transit', 
#                     # 'Liquor Store Revenue',
#                 #    'Insurance Trust Revenue',
#                 #    'Unemployment Compensation', 'Employee Retirement', "Workers' Compensation", 'Other Insurance Trust Revenue']}
# figs = {}
# for key in plot_dfs.keys():
#     figs[key] = {}
#     for form, plot_df in plot_dfs[key].items():# ["Level", "Real Level", "Percent of General Revenue", "Percent of GDP", "Real Value Per Capita"]:
#         figs[key][form] = {}
#         df = plot_df.copy()
#         df = df.reset_index().melt(id_vars=["State","Year"],
#             var_name="Name", 
#             value_name="Value")
#         df = df.set_index(["State", "Year"]).round(2)
#         states = df.index.get_level_values("State").unique()

#         for components_group, components in areas.items():


#             figs[key][form][components_group] = {}

#             for state in states:
#                 plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
#                 plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
#                 plot_df["Value"] = pd.to_numeric(plot_df["Value"])
#                 title_key = f"{components_group} {key}<br>{form}<br>{state}"
#                 px_fig = px.area(
#                     plot_df, x="Year", y="Value", color="Name", title = title_key)

#                 figs[key][form][components_group][state] = px_fig
#             figs[key][form][components_group]  = dict_of_figs_to_dropdown_fig(figs[key][form][components_group], 
#                                                                               show_fig = False,
#                                                                               use_sliders = True)
#             figs[key][form][components_group].write_html(f"outputs/{key}/AreaPlots{key}{form}{components_group}Figs.html")

In [22]:
# spliced_revenue = pd.read_csv("SplicedStateGovFinances.csv")
# spliced_revenue.set_index(["State", "Year"]).sort_index()

In [23]:
# import os
# folder = "State government amount"
# frmt = "LevelDiscrepancies"
# regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
# # scatter_figs = {}
# df = diff_df.copy()
# map_figs = {}
# create_scatter_dropdown(df, regions_df=regions_df,
#                         filename = f"outputs/{folder}/ScatterPlotsIncomePropertyAssessmentSalesFuelTaxesPctTotalRevenue{folder}{frmt}.html", 
#                         show_fig = False)
# for name in df.keys():
#     map_figs[name] = create_map(df.reset_index(), name, time_name = "Year")
# combined_map_fig = combine_map_figs(map_figs)
# combined_map_fig.write_html(f"outputs/{folder}/MapPlotsByVariableAndYear{folder}{frmt}.html")

#     # combined_scatter_fig = dict_of_figs_line_figs_to_dropdown_fig(scatter_figs,regions_df)


# fig = line_dropdown(df, regions_df)
# # fig.show()
# fig.write_html(f"outputs/{folder}/LinePlotsStateFinancesAsPercentRevenuePercentGDPAndPerCapitaFigs{folder}{frmt}.html")